In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Inspect datasets

In [ ]:
import pandas as pd

train = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv")

train.tail()

## Load image and label data

In [ ]:
x_train = train.iloc[:, 1:].to_numpy()
y_train = train.iloc[:, 0].to_numpy()

print(x_train.shape)
print(y_train.shape)

## Plot an image

In [ ]:
import matplotlib.pyplot as plt

one_image = x_train[42]

plt.imshow(one_image.reshape(28, 28), cmap = "gray")
# plt.imshow(one_image.reshape(28, 28, 1), cmap = "gray") also works
print(f'The label for this image is : {y_train[42]}')
plt.matshow(one_image.reshape(28, 28), cmap = "gray") # same output!

# Baseline Model - logistic regression

In [ ]:
# scale and convert dtype

x_train = x_train/255.
x_train = x_train.astype("float32")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

mlp = Sequential([
    Dense(10, input_shape = (784, ), activation = "softmax")
])

mlp.compile(loss = "sparse_categorical_crossentropy",
            optimizer = "sgd",
            metrics = ["accuracy"])

history = mlp.fit(x_train, y_train, epochs = 15, validation_split = 0.1)

In [ ]:
train_acc = history.history["accuracy"]
train_loss = history.history["loss"]
val_acc = history.history["val_accuracy"]
val_loss = history.history["val_loss"]

epochs = range(len(train_acc))

plt.plot(epochs, train_acc, "o-g", label = "Train")
plt.plot(epochs, val_acc, "o-b", label = "Validation")
plt.title("Accuracy")
plt.legend()
plt.figure()

plt.plot(epochs, train_loss, "o-g", label = "Train")
plt.plot(epochs, val_loss, "o-b", label = "Validation")
plt.title("Loss")
plt.legend()
plt.figure()

## Simple CNN

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)
print(x_train.shape)
print(x_train.dtype)

In [ ]:
simple_model = Sequential([
                Conv2D(filters = 32, kernel_size = (5, 5), padding = "same", input_shape = (28, 28, 1),
                      activation = "relu"),
                MaxPooling2D(),
                Conv2D(filters = 64, kernel_size = (3, 3), padding = "same",
                      activation = "relu"),
                MaxPooling2D(),
                Flatten(),
                Dense(256, activation = "relu"),
                Dense(10, activation = "softmax")
])

simple_model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

history = simple_model.fit(x_train, y_train, epochs = 10, validation_split = 0.1)

## Build a better model 

## Early Stopping Callback

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', patience = 3) 
# patience: Number of epochs with no improvement after which training will be stopped.

In [ ]:
fancy_cnn = Sequential([
    Conv2D(filters = 32, kernel_size = (3, 3), padding = "same", activation = "relu", input_shape = (28, 28, 1)),
    MaxPooling2D(),
    Conv2D(filters = 64, kernel_size = (3, 3), padding = "same", activation = "relu", input_shape = (28, 28, 1)),
    MaxPooling2D(),
    Flatten(),
    Dense(50, activation = "relu"),
    Dropout(0.5),
    Dense(10, activation = "softmax")
])

fancy_cnn.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

history = fancy_cnn.fit(x_train, y_train, epochs = 20, validation_split = 0.1,
                   callbacks = [es])

# Evaluate using test set

In [ ]:
dig_mnist = pd.read_csv("/kaggle/input/Kannada-MNIST/Dig-MNIST.csv")

x_test = dig_mnist.iloc[:, 1:].to_numpy()
y_test = dig_mnist.iloc[:, 0].to_numpy()

print(x_test.shape)
print(y_test.shape)

In [ ]:
x_test = x_test/255.

x_test = x_test.reshape(-1, 28, 28, 1).astype("float32")
print(x_test.shape)

In [ ]:
one_image = x_test[0]
plt.imshow(one_image.squeeze(), cmap = "gray")
print(y_test[0])

In [ ]:
fancy_cnn.predict(one_image.reshape(1, 28, 28, 1))

In [ ]:
fancy_cnn.evaluate(x_test, y_test)

## Make predictions on real test set and submit

In [ ]:
real_test = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
real_test.head()
real_test = real_test.iloc[:, 1:].to_numpy()
real_test = real_test/255.
real_test.shape
real_test_2d = real_test.reshape(-1, 28, 28, 1)

In [ ]:
sample_submit = pd.read_csv("/kaggle/input/Kannada-MNIST/sample_submission.csv")
sample_submit.head()

# test = pd.read_csv("/kaggle/input/Kannada-MNIST/test.csv")
# test.head()

In [ ]:
predictions = fancy_cnn.predict(real_test_2d)
predictions.shape

sample_submit["label"] = predictions.argmax(axis = 1)

In [ ]:
sample_submit.to_csv("submission.csv", index = False)